## Compute NDCG Scores

In [ ]:
import csv
from models.rank_eval import get_ndcg

## Write functions to read two different csv files - scored-results.csv and question-results.csv

In [ ]:
# Function to read the first CSV file with "Question," "ResultId," and "Score" columns
def read_true_csv(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            question = row.get('Question', '')  # Use get to handle missing 'Question' key
            result_id = row.get('ResultId', '')  # Use get to handle missing 'ResultId' key
            score = float(row.get('Score', 0))  # Use get to handle missing 'Score' key and convert to float
            if question and result_id:  # Check if both 'Question' and 'ResultId' are present
                if question not in data_dict:
                    data_dict[question] = []
                data_dict[question].append({'id': result_id, 'score': score})
    return data_dict

# Function to read the second CSV file with "Question," "ResultId," and "ResultRank" columns
def read_predicted_csv(file_path):
    data_dict = {}
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            question = row.get('Question', '')  # Use get to handle missing 'Question' key
            result_id = row.get('ResultId', '')  # Use get to handle missing 'ResultId' key
            result_rank = int(row.get('ResultRank', 0))  # Use get to handle missing 'ResultRank' key and convert to int
            if question and result_id:  # Check if both 'Question' and 'ResultId' are present
                if question not in data_dict:
                    data_dict[question] = []
                data_dict[question].append({'id': result_id, 'score': result_rank})
    return data_dict



## Write a function to compute the NDCG scores

In [ ]:
# Function to calculate NDCG scores
def calculate_ndcg(true_results, predicted_results, k_values):
    ndcg_scores = {}
    single_document_questions = []  # Keep track of questions with only one document
    
    for question in true_results:
        true_list = true_results[question]
        predicted_list = predicted_results.get(question, [])
        
        if len(true_list) == 1:
            # Append details of questions with only one document
            single_document_questions.append({
                'question': question,
                'true_result': true_list[0],
                'predicted_result': predicted_list[0] if predicted_list else None
            })
        else:
            # Calculate NDCG scores for questions with more than one document
            ndcg_scores[question] = [get_ndcg(true_list, predicted_list, k) for k in k_values]
    
    # Print questions with only one document
    print("Questions with Only One Document:")
    for item in single_document_questions:
        print(f"Question: {item['question']}")
        print(f"True Result: {item['true_result']}")
        print(f"Predicted Result: {item['predicted_result']}")
        print()
    
    return ndcg_scores

## Call the functions to read the csv files and to compute NDCG scores

In [ ]:
# Define file paths
true_results_file = '../references/scored-results.csv'
predicted_results_file = '../data/exports/question-results.csv'
    
# Read the first CSV file with true results
true_results = read_true_csv(true_results_file)

# Read the second CSV file with predicted results
predicted_results = read_predicted_csv(predicted_results_file)

# Calculate NDCG scores
k_values = [5, 3, 10]
ndcg_scores = calculate_ndcg(true_results, predicted_results, k_values)
# print("predicted results", predicted_results)
# print()
# print("true results", true_results)
# Print NDCG scores
print("NDCG Scores:")
for question, scores in ndcg_scores.items():
    print(f"Question: {question}, NDCG Scores: {scores}")

In [ ]:
len(ndcg_scores)

In [ ]:
# Calculate and print average NDCG scores
avg_ndcg_scores = {k: sum(scores) / len(scores) for k, scores in zip(k_values, zip(*ndcg_scores.values()))}
print("\nAverage NDCG Scores:")
for k, avg_score in avg_ndcg_scores.items():
    print(f"At k={k}: {avg_score}")